In [6]:
[jsonparse].

true

In [31]:
string_tokens('[]{}   ', X).

X = [OPENBRACKET,CLOSEDBRACKET,OPENCURLY,CLOSEDCURLY]

In [ ]:
jsonread("in.txt", X).

In [ ]:
jsonread("in.txt", X), term_json(X,Y).

In [33]:
%jsondump(J, S)/2

jsondump(J, S) :-
    json_string_(J, S , 0, 0, Rest).


Previously defined clauses of user:jsondump/2 were retracted:
:- dynamic jsondump/2.

jsondump(A, B) :-
    json_string_(A, B, 0, 0, _).

% Asserting clauses for user:jsondump/2


Previously defined clauses of user:json_string_indent/5 were retracted:
:- dynamic json_string_indent/5.

json_string_indent(_, _, _, _, []).

% Asserting clauses for user:json_string_indent/5


Previously defined clauses of user:json_string_/5 were retracted:
:- dynamic json_string_/5.

json_string_([j, s, o, n, o, b, j, '(', '[', ']', ')'|A], "{}", _, _, A).
json_string_([j, s, o, n, a, r, r, a, y, '(', '[', ']', ')'|A], "[]", _, _, A).

% Asserting clauses for user:json_string_/5


In [3]:
%prefix_list_rest/3 takes a prefix as a string
% checks if it's a prefix of a list of chars
prefix_list_rest(String, L, R):-
    atom_chars(String, Cs),
    append(Cs, R, L).
    

Previously defined clauses of user:prefix_list_rest/3 were retracted:
:- dynamic prefix_list_rest/3.

prefix_list_rest(A, B, C) :-
    atom_chars(A, D),
    append(D, B, C).

% Asserting clauses for user:prefix_list_rest/3


In [4]:
prefix_list_rest("c", ['c', 'i', 'a'], X).

X = [i,a]

In [53]:
%json_string_/5 %% JsonObj, String, OldIndent, NewIndent Rest

%% empty rest, basecase
json_string_indent(_, _, _, _, []).

json_string_([O | Js], `[]`, OldI, NewI, Js):-
    atom_chars("jsonarray([" , A),
    append(A, B, [']',')']).

Previously defined clauses of user:json_string_indent/5 were retracted:
:- dynamic json_string_indent/5.

json_string_indent(_, _, _, _, []).

% Asserting clauses for user:json_string_indent/5


Previously defined clauses of user:json_string_/5 were retracted:
:- dynamic json_string_/5.

json_string_([_|A], "[]", _, _, A) :-
    atom_chars("jsonarray([", B),
    append(B, _, [']', ')']).

% Asserting clauses for user:json_string_/5


In [ ]:
json_string_(`jsonarray([])`, S , 0, 0, R).

false

In [42]:
atom_chars("", X).

X = []

In [38]:
jsonparse('[1,2,3]', X),
json_chars(X, C),
atom_chars("jsonarray([",A),
append(A,B, C).

X = jsonarray([1,2,3]),
C = [j,s,o,n,a,r,r,a,y,(,[,1,,,2,,,3,],)],
A = [j,s,o,n,a,r,r,a,y,(,[],
B = [1,,,2,,,3,],)]

In [ ]:
jsonparse('[]', X),
json_chars(X, R1),
write(R1),
json_string_(R1, S, I, NI, R).

[j,s,o,n,a,r,r,a,y,(,[,1,,,2,,,3,],)]

false

In [ ]:
json_string_([j,s,o,n,o,b,j,'('], S, I, I, Rest).

In [4]:
json_chars(J, C):-
    term_to_atom(J, A),
    atom_chars(A, C).

% Asserting clauses for user:json_chars/2


In [ ]:
term_json(Term, Json) :-
    phrase(parse_object(Term), JsonChars),
    atom_chars(Json, JsonChars).
    
parse_object(jsonobj([])) -->
    !,
    ['{'],
    ['}'].
    
parse_object(jsonobj(Members)) -->
    ['{'],
    parse_members(Members),
    ['}'].

% members
parse_members([Pair]) -->
    !,
    parse_pair(Pair).

parse_members([Pair|Pairs]) -->
    parse_pair(Pair),
    [','],
    parse_members(Pairs).

%pair
parse_pair((Key, Value)) -->
    parse_key(Key),
    [':'],
    parse_value(Value).

parse_key(Key) -->
    parse_atom(Key).

parse_value(jsonarray(Value)) -->
    %{ util:looks_like_list(Value) }, % Must precede atom/1 check!
    !,
    parse_array(Value).

parse_value(json(Value)) -->
    !,
    parse_object(json(Value)).

parse_value(+Value) -->
    !,
    parse_symbol(Value).
    
parse_value(Value) -->
    { core:atom(Value) },
    !,
    parse_atom(Value).
    
parse_value(Value) -->
    { core:number(Value) },
    !,
    parse_number(Value).

parse_atom(Atom) -->
    ['"'],
    { core:atom_chars(Atom, Chars) },
    parse_string_chars(Chars),
    ['"'].

parse_string_chars([]) --> !, [].
parse_string_chars(Chars) -->
    parse_special_chars(Chars, Chars1),
    !,
    parse_string_chars(Chars1).

parse_string_chars([Char|Chars]) -->
    [Char],
    parse_string_chars(Chars).

parse_special_chars(['\\'|Chars], Chars1) -->
    ['\\'],
    !,
    parse_escape_sequence(Chars, Chars1).
parse_special_chars([Char|Chars], Chars) -->
    { single_special_char(Char, EscapedChar) },
    ['\\',EscapedChar].

single_special_char('"',  '"').
single_special_char('/',  '/').
single_special_char('\b', 'b').
single_special_char('\f', 'f').
single_special_char('\n', 'n').
single_special_char('\r', 'r').

parse_escape_sequence(['u',Hex1,Hex2,Hex3,Hex4|Chars], Chars) -->
    ['u',Hex1,Hex2,Hex3,Hex4],
    !.
parse_escape_sequence(Chars, Chars) -->
    ['\\'].

parse_number(Number) -->
    { core:number_chars(Number, Chars) },
    Chars.

parse_array([]) -->
    !,
    ['['],
    [']'].
parse_array(Values) -->
    ['['],
    parse_array_values(Values),
    [']'].

parse_array_values([Value]) -->
    !,
    parse_value(Value).
parse_array_values([Value|Values]) -->
    parse_value(Value),
    [','],
    parse_array_values(Values).

parse_symbol(true)  --> !, [t,r,u,e].
parse_symbol(false) --> !, [f,a,l,s,e].
parse_symbol(null)  --> !, [n,u,l,l].

looks_like_list([]).
looks_like_list([_|_]).

In [ ]:
term_json(Term, Json) :-
    phrase(parse_object(Term), JsonChars),
    atom_chars(Json, JsonChars).
    
parse_object(jsonobj([])) -->
    !,
    ['{'],
    ['}'].
    
parse_object(jsonobj(Members)) -->
    ['{'],
    parse_members(Members),
    ['}'].

% members
parse_members([Pair]) -->
    !,
    parse_pair(Pair).

parse_members([Pair|Pairs]) -->
    parse_pair(Pair),
    [','],
    parse_members(Pairs).

%pair
parse_pair((Key, Value)) -->
    parse_key(Key),
    [':'],
    parse_value(Value).

parse_key(Key) -->
    parse_atom(Key).

parse_value(jsonarray(Value)) -->
    { util:looks_like_list(Value) }, % Must precede atom/1 check!
    !,
    parse_array(Value).

parse_value(json(Value)) -->
    !,
    parse_object(json(Value)).

parse_value(+Value) -->
    !,
    parse_symbol(Value).
    
parse_value(Value) -->
    { core:atom(Value) },
    !,
    parse_atom(Value).
    
parse_value(Value) -->
    { core:number(Value) },
    !,
    parse_number(Value).

parse_atom(Atom) -->
    ['"'],
    { core:atom_chars(Atom, Chars) },
    parse_string_chars(Chars),
    ['"'].

parse_string_chars([]) --> !, [].
parse_string_chars(Chars) -->
    parse_special_chars(Chars, Chars1),
    !,
    parse_string_chars(Chars1).

parse_string_chars([Char|Chars]) -->
    [Char],
    parse_string_chars(Chars).

parse_special_chars(['\\'|Chars], Chars1) -->
    ['\\'],
    !,
    parse_escape_sequence(Chars, Chars1).
parse_special_chars([Char|Chars], Chars) -->
    { single_special_char(Char, EscapedChar) },
    ['\\',EscapedChar].

single_special_char('"',  '"').
single_special_char('/',  '/').
single_special_char('\b', 'b').
single_special_char('\f', 'f').
single_special_char('\n', 'n').
single_special_char('\r', 'r').

parse_escape_sequence(['u',Hex1,Hex2,Hex3,Hex4|Chars], Chars) -->
    ['u',Hex1,Hex2,Hex3,Hex4],
    !.
parse_escape_sequence(Chars, Chars) -->
    ['\\'].

parse_number(Number) -->
    { core:number_chars(Number, Chars) },
    Chars.

parse_array([]) -->
    !,
    ['['],
    [']'].
parse_array(Values) -->
    ['['],
    parse_array_values(Values),
    [']'].

parse_array_values([Value]) -->
    !,
    parse_value(Value).
parse_array_values([Value|Values]) -->
    parse_value(Value),
    [','],
    parse_array_values(Values).

parse_symbol(true)  --> !, [t,r,u,e].
parse_symbol(false) --> !, [f,a,l,s,e].
parse_symbol(null)  --> !, [n,u,l,l].

looks_like_list([]).
looks_like_list([_|_]).